In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
import json
import plotly.express as px
import pandas as pd
import folium
import streamlit as st
import folium
from streamlit_folium import st_folium
from shapely.geometry import box
# pd.set_option('display.max_rows', None)
from folium.plugins import HeatMap, TimestampedGeoJson
import branca.colormap as cm
import numpy as np


In [4]:
df = pd.read_csv('app/datameteo_france_1950-2022_clean_02.csv')
df.head()


# Filter the DataFrame for the specific Year and Month
df_filtered = df[(df['Year'] == 2012) & (df['Month'] == 1)]

# Create a map centered around the mean latitude and longitude of the filtered DataFrame
map_center = [df_filtered['LAT'].mean(), df_filtered['LON'].mean()]
mymap = folium.Map(location=map_center, zoom_start=4)

# Prepare data for the HeatMap (latitude, longitude, and wind speed as intensity)
heat_data = [[row['LAT'], row['LON'], row['vent_speed_inst_moy_mensu']] for index, row in df_filtered.iterrows()]

# Define the colormap
colormap = cm.LinearColormap(colors=['#5F97CF', '#90E1A9', '#F4EB87', '#FFD391', '#D85356'], 
                             vmin=min(df_filtered['vent_speed_inst_moy_mensu']), 
                             vmax=max(df_filtered['vent_speed_inst_moy_mensu']))


# radius: Reduces the size of each data point. Smaller radius limits the overlap of color intensity.
# blur: Controls how much the color intensity of each point spreads out. Lower blur values can help reduce the excessive blending of colors.

heatmap_group = folium.FeatureGroup(name='Wind Speed Heatmap', overlay=True, show=True)

# Adjust HeatMap parameters
HeatMap(
    heat_data,
    min_opacity=0.01,  # Set lower opacity to reduce intensity
    max_val=max(df_filtered['vent_speed_inst_moy_mensu']),
    radius=20,  # Reduce radius to lessen the overlap when zooming out
    blur=10,  # Reduce blur for better definition
).add_to(heatmap_group)



# Add the colormap (color scale) to the map
colormap.caption = 'Wind Speed (m/s)'
colormap.add_to(mymap)


heatmap_group.add_to(mymap)


wind_direction_group = folium.FeatureGroup(name='Wind Direction', overlay=True, show=False)


for idx, row in df_filtered.iterrows():
    wind_direction = row['vent_dir_inst']
    
    # Unique animation name for each arrow
    animation_name = f"moveAndRotate_{idx}"
    
    # Customize these values to adjust animation
    duration = "1s"  # Change to control speed (e.g., "1s", "3s")
    amplitude = 6  # Distance the arrow moves, change as needed
    timing_function = "ease-in-out"  # Options: linear, ease, ease-in, ease-out, ease-in-out

    # Generate unique keyframes animation
    icon_html = f"""
    <div style="
        font-size: 15px;
        color: #183153;
        position: relative;
        display: flex;
        justify-content: center;
        align-items: center;
        width: 15px;
        height: 15px;
        animation: {animation_name} {duration} infinite alternate {timing_function};
    ">
        &rarr;
</div>
    <style>
        @keyframes {animation_name} {{
            0% {{ transform: translate(0, 0) rotate({wind_direction}deg); }}
            100% {{ transform: translate({amplitude * np.cos(np.radians(wind_direction))}px, {amplitude * np.sin(np.radians(wind_direction))}px) rotate({wind_direction}deg); }}
        }}
    </style>
    """
    folium.Marker(
        location=[row['LAT'], row['LON']],
        icon=folium.DivIcon(html=icon_html),
        tooltip=f"Wind Speed: {row['vent_speed_inst_moy_mensu']} m/s"
    ).add_to(wind_direction_group)


wind_direction_group.add_to(mymap)

folium.LayerControl().add_to(mymap)

# Display the map
mymap

C:\Users\j3r3m\AppData\Local\Temp\ipykernel_18500\1253656001.py:27: UserWarning: The `max_val` parameter is no longer necessary. The largest intensity is calculated automatically.
  HeatMap(
